# Experiment 003: Simulated Annealing from Scratch

The baseline is at a local optimum. Simple local search cannot improve it.
We need global optimization to escape the local optimum.

This experiment implements simulated annealing in pure Python.

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union
import json
import math
import random
import time
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Tree shape vertices
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

print("Setup complete")

Setup complete


In [2]:
# Core functions
def create_tree_polygon(x, y, angle):
    """Create a tree polygon at position (x, y) with given rotation angle."""
    x, y, angle = float(x), float(y), float(angle)
    coords = list(zip(TX, TY))
    poly = Polygon(coords)
    poly = affinity.rotate(poly, angle, origin=(0, 0))
    poly = affinity.translate(poly, x, y)
    return poly

def create_scaled_tree_polygon(x, y, angle, scale_factor=1e15):
    """Create tree polygon with scaling for strict precision."""
    x, y, angle = float(x), float(y), float(angle)
    coords = [(tx * scale_factor, ty * scale_factor) for tx, ty in zip(TX, TY)]
    poly = Polygon(coords)
    poly = affinity.rotate(poly, angle, origin=(0, 0))
    poly = affinity.translate(poly, xoff=x * scale_factor, yoff=y * scale_factor)
    return poly

def get_bbox_side(trees):
    """Get bounding box side length for a list of trees."""
    if len(trees) == 0:
        return 0
    polygons = [create_tree_polygon(t['x'], t['y'], t['deg']) for t in trees]
    union = unary_union(polygons)
    bounds = union.bounds
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    return max(width, height)

def has_overlap(trees):
    """Check for overlaps using strict precision."""
    if len(trees) <= 1:
        return False
    polygons = [create_scaled_tree_polygon(t['x'], t['y'], t['deg']) for t in trees]
    n = len(polygons)
    for i in range(n):
        for j in range(i+1, n):
            if polygons[i].intersects(polygons[j]):
                if not polygons[i].touches(polygons[j]):
                    intersection = polygons[i].intersection(polygons[j])
                    if intersection.area > 0:
                        return True
    return False

def parse_value(val):
    """Parse value from submission format."""
    if isinstance(val, str) and val.startswith('s'):
        return val[1:]
    return str(val)

print("Core functions defined")

Core functions defined


In [3]:
# Simulated Annealing implementation
def perturb(trees, perturbation_scale=0.1):
    """Randomly perturb one tree's position or rotation."""
    result = [dict(t) for t in trees]
    idx = random.randint(0, len(result) - 1)
    
    # Choose perturbation type
    choice = random.random()
    if choice < 0.4:
        # Move tree in x
        dx = random.uniform(-perturbation_scale, perturbation_scale)
        result[idx]['x'] = str(float(result[idx]['x']) + dx)
    elif choice < 0.8:
        # Move tree in y
        dy = random.uniform(-perturbation_scale, perturbation_scale)
        result[idx]['y'] = str(float(result[idx]['y']) + dy)
    else:
        # Rotate tree
        dangle = random.uniform(-15, 15)
        result[idx]['deg'] = str(float(result[idx]['deg']) + dangle)
    
    return result

def simulated_annealing(trees, T_init=1.0, T_min=0.0001, cooling=0.9995, max_iter=50000, verbose=False):
    """Simulated annealing for a single N configuration."""
    current = [dict(t) for t in trees]
    current_score = get_bbox_side(current)
    best = [dict(t) for t in current]
    best_score = current_score
    T = T_init
    
    accepted = 0
    rejected = 0
    improved = 0
    
    for iteration in range(max_iter):
        # Adaptive perturbation scale based on temperature
        perturbation_scale = 0.1 * (T / T_init) + 0.01
        
        # Random perturbation
        candidate = perturb(current, perturbation_scale)
        
        # Check for overlaps
        if has_overlap(candidate):
            rejected += 1
            continue
        
        candidate_score = get_bbox_side(candidate)
        delta = candidate_score - current_score
        
        # Accept if better, or with probability exp(-delta/T) if worse
        if delta < 0:
            # Better solution - always accept
            current = [dict(t) for t in candidate]
            current_score = candidate_score
            accepted += 1
            if current_score < best_score:
                best = [dict(t) for t in current]
                best_score = current_score
                improved += 1
        elif T > 0 and random.random() < math.exp(-delta / T):
            # Worse solution - accept with probability
            current = [dict(t) for t in candidate]
            current_score = candidate_score
            accepted += 1
        else:
            rejected += 1
        
        T *= cooling
        if T < T_min:
            break
        
        if verbose and iteration % 10000 == 0:
            print(f"  Iter {iteration}: T={T:.6f}, current={current_score:.6f}, best={best_score:.6f}")
    
    return best, best_score, {'accepted': accepted, 'rejected': rejected, 'improved': improved}

print("Simulated annealing function defined")

Simulated annealing function defined


In [4]:
# Load baseline submission
df = pd.read_csv('/home/code/experiments/001_fix_overlaps/submission.csv')
print(f"Loaded baseline with {len(df)} rows")

# Parse into structured format
trees_by_n = {}
for _, row in df.iterrows():
    id_parts = row['id'].split('_')
    n = int(id_parts[0])
    idx = int(id_parts[1])
    
    if n not in trees_by_n:
        trees_by_n[n] = []
    
    trees_by_n[n].append({
        'idx': idx,
        'x': parse_value(row['x']),
        'y': parse_value(row['y']),
        'deg': parse_value(row['deg'])
    })

print(f"Parsed trees for N=1 to {max(trees_by_n.keys())}")

# Calculate baseline scores
baseline_per_n = {}
for n in range(1, 201):
    side = get_bbox_side(trees_by_n[n])
    baseline_per_n[n] = (side ** 2) / n

baseline_total = sum(baseline_per_n.values())
print(f"Baseline total score: {baseline_total:.6f}")

Loaded baseline with 20100 rows


Parsed trees for N=1 to 200


Baseline total score: 70.622435


In [5]:
# Test SA on a few N values first
print("=" * 60)
print("TESTING SIMULATED ANNEALING ON SMALL N VALUES")
print("=" * 60)

test_ns = [10, 20, 30]
test_results = {}

for n in test_ns:
    print(f"\nTesting N={n}...")
    trees = trees_by_n[n]
    baseline_side = get_bbox_side(trees)
    baseline_score = (baseline_side ** 2) / n
    
    # Run SA with moderate parameters
    start_time = time.time()
    best_trees, best_side, stats = simulated_annealing(
        trees, 
        T_init=0.5,  # Start with moderate temperature
        T_min=0.0001,
        cooling=0.9998,  # Slow cooling
        max_iter=30000,
        verbose=True
    )
    elapsed = time.time() - start_time
    
    best_score = (best_side ** 2) / n
    improvement = baseline_score - best_score
    
    test_results[n] = {
        'baseline_side': baseline_side,
        'best_side': best_side,
        'baseline_score': baseline_score,
        'best_score': best_score,
        'improvement': improvement,
        'stats': stats,
        'time': elapsed
    }
    
    print(f"  Baseline: side={baseline_side:.6f}, score={baseline_score:.6f}")
    print(f"  SA Best:  side={best_side:.6f}, score={best_score:.6f}")
    print(f"  Improvement: {improvement:.6f} ({improvement/baseline_score*100:.2f}%)")
    print(f"  Time: {elapsed:.1f}s, Accepted: {stats['accepted']}, Rejected: {stats['rejected']}")

TESTING SIMULATED ANNEALING ON SMALL N VALUES

Testing N=10...


  Iter 10000: T=0.082635, current=3.671526, best=1.940696


  Iter 20000: T=0.012531, current=3.205060, best=1.940696


  Baseline: side=1.940696, score=0.376630
  SA Best:  side=1.940696, score=0.376630
  Improvement: 0.000000 (0.00%)
  Time: 59.8s, Accepted: 26297, Rejected: 3703

Testing N=20...


  Iter 10000: T=0.108446, current=4.408068, best=2.742469


  Iter 20000: T=0.017421, current=4.343788, best=2.742469


  Baseline: side=2.742469, score=0.376057
  SA Best:  side=2.742469, score=0.376057
  Improvement: 0.000000 (0.00%)
  Time: 133.1s, Accepted: 25391, Rejected: 4609

Testing N=30...


  Iter 10000: T=0.134434, current=5.423509, best=3.290365


  Iter 20000: T=0.022036, current=5.241542, best=3.290365


  Baseline: side=3.290365, score=0.360883
  SA Best:  side=3.290365, score=0.360883
  Improvement: 0.000000 (0.00%)
  Time: 217.1s, Accepted: 24474, Rejected: 5526


In [ ]:
# Analyze test results
print("\n" + "=" * 60)
print("TEST RESULTS SUMMARY")
print("=" * 60)

total_improvement = 0
for n, result in test_results.items():
    print(f"N={n}: improvement = {result['improvement']:.6f}")
    total_improvement += result['improvement']

print(f"\nTotal improvement on test N values: {total_improvement:.6f}")

if total_improvement > 0:
    print("\n✅ SA shows promise! Running on all N values...")
else:
    print("\n⚠️ SA didn't improve test N values. Trying different parameters...")

In [ ]:
# Run SA on ALL N values with optimized parameters
print("\n" + "=" * 60)
print("RUNNING SA ON ALL N VALUES")
print("=" * 60)

optimized_trees_by_n = {}
improvements = []

start_total = time.time()

for n in range(1, 201):
    trees = trees_by_n[n]
    baseline_side = get_bbox_side(trees)
    baseline_score = (baseline_side ** 2) / n
    
    if n == 1:
        # N=1 is already optimal at 45°
        optimized_trees_by_n[n] = [dict(t) for t in trees]
        continue
    
    # Adjust iterations based on N (more trees = more iterations needed)
    max_iter = min(20000 + n * 100, 50000)
    
    # Run SA
    best_trees, best_side, stats = simulated_annealing(
        trees,
        T_init=0.3,
        T_min=0.0001,
        cooling=0.9997,
        max_iter=max_iter,
        verbose=False
    )
    
    best_score = (best_side ** 2) / n
    improvement = baseline_score - best_score
    
    if improvement > 1e-8:
        optimized_trees_by_n[n] = best_trees
        improvements.append((n, improvement))
        print(f"N={n:3d}: {baseline_score:.6f} -> {best_score:.6f} (+{improvement:.6f})")
    else:
        optimized_trees_by_n[n] = [dict(t) for t in trees]  # Keep baseline
    
    # Progress update every 20 N values
    if n % 20 == 0:
        elapsed = time.time() - start_total
        print(f"  Progress: N={n}/200, elapsed={elapsed:.1f}s")

print(f"\nTotal time: {time.time() - start_total:.1f}s")
print(f"N values improved: {len(improvements)}")

In [ ]:
# Calculate final score
optimized_per_n = {}
for n in range(1, 201):
    side = get_bbox_side(optimized_trees_by_n[n])
    optimized_per_n[n] = (side ** 2) / n

optimized_total = sum(optimized_per_n.values())

print("\n" + "=" * 60)
print("FINAL RESULTS")
print("=" * 60)
print(f"Baseline score: {baseline_total:.6f}")
print(f"Optimized score: {optimized_total:.6f}")
print(f"Total improvement: {baseline_total - optimized_total:.6f}")
print(f"Target: 68.888293")
print(f"Gap to target: {optimized_total - 68.888293:.6f}")

if improvements:
    print(f"\nN values improved: {len(improvements)}")
    print("Top 10 improvements:")
    for n, imp in sorted(improvements, key=lambda x: -x[1])[:10]:
        print(f"  N={n}: +{imp:.6f}")

In [ ]:
# Validate no overlaps
print("\n" + "=" * 60)
print("VALIDATION")
print("=" * 60)

overlap_errors = []
for n in range(1, 201):
    trees = optimized_trees_by_n[n]
    if n > 1 and has_overlap(trees):
        overlap_errors.append(n)
        print(f"  ⚠️ N={n}: Has overlaps")

if overlap_errors:
    print(f"\n❌ {len(overlap_errors)} N values have overlaps!")
else:
    print("\n✅ All N values pass overlap validation!")

In [ ]:
# Create submission
print("\n" + "=" * 60)
print("CREATING SUBMISSION")
print("=" * 60)

rows = []
for n in range(1, 201):
    trees = optimized_trees_by_n[n]
    for i, t in enumerate(trees):
        rows.append({
            'id': f"{n:03d}_{i}",
            'x': f"s{t['x']}",
            'y': f"s{t['y']}",
            'deg': f"s{t['deg']}"
        })

submission_df = pd.DataFrame(rows)
print(f"Submission shape: {submission_df.shape}")

submission_df.to_csv('/home/code/experiments/003_simulated_annealing/submission.csv', index=False)
submission_df.to_csv('/home/submission/submission.csv', index=False)
print("Submission saved!")

In [ ]:
# Save metrics
metrics = {
    'cv_score': optimized_total,
    'baseline_score': baseline_total,
    'improvement': baseline_total - optimized_total,
    'n_values_improved': len(improvements),
    'overlap_errors': len(overlap_errors),
    'target': 68.888293,
    'gap': optimized_total - 68.888293
}

with open('/home/code/experiments/003_simulated_annealing/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("\nMetrics saved!")
print(f"Final CV Score: {optimized_total:.6f}")

In [ ]:
# SA test results show no improvement - the baseline is at a very tight local optimum\n# Let's try a different approach: generate random initial configurations\n\nprint(\"\\n\" + \"=\" * 60)\nprint(\"SA TEST RESULTS: NO IMPROVEMENT\")\nprint(\"=\" * 60)\nprint(\"\"\"\\nThe simulated annealing test shows that even with 30,000 iterations,\nSA couldn't improve any of the test N values (N=10, 20, 30).\n\nThis confirms that the baseline is at a VERY TIGHT local optimum.\nThe C++ optimizers (bbox3, etc.) have already found excellent solutions.\n\nConclusion: We cannot beat the baseline with simple optimization.\nThe gap of 1.73 points requires fundamentally different approaches.\n\"\"\")\n\n# Since SA didn't help, let's just use the baseline\noptimized_trees_by_n = {n: [dict(t) for t in trees] for n, trees in trees_by_n.items()}\noptimized_total = baseline_total\nimprovements = []\noverlap_errors = []"}